# Bert tutorial using huggingface pytorch library

In [3]:
import torch
from transformers import (pipeline, BertTokenizer, BertModel, BertConfig,
                          BertForNextSentencePrediction,
                          BertForSequenceClassification,
                          BertForTokenClassification,
                          BertForQuestionAnswering
                         )

In [ ]:
pipeline("ner")

I0525 06:46:08.195096 139846855460672 filelock.py:274] Lock 139842528611016 acquired on /home/haridas/.cache/torch/transformers/90deb4d9dd705272dc4b3db1364d759d551d72a9f70a91f60e3a1f5e278b985d.9019d8d0ae95e32b896211ae7ae130d7c36bb19ccf35c90a9e51923309458f70.lock
I0525 06:46:08.197701 139846855460672 file_utils.py:436] https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-cased-config.json not found in cache or force_download set to True, downloading to /home/haridas/.cache/torch/transformers/tmpw86204qi


I0525 06:46:09.693573 139846855460672 file_utils.py:440] storing https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-cased-config.json in cache at /home/haridas/.cache/torch/transformers/90deb4d9dd705272dc4b3db1364d759d551d72a9f70a91f60e3a1f5e278b985d.9019d8d0ae95e32b896211ae7ae130d7c36bb19ccf35c90a9e51923309458f70
I0525 06:46:09.695026 139846855460672 file_utils.py:443] creating metadata file for /home/haridas/.cache/torch/transformers/90deb4d9dd705272dc4b3db1364d759d551d72a9f70a91f60e3a1f5e278b985d.9019d8d0ae95e32b896211ae7ae130d7c36bb19ccf35c90a9e51923309458f70
I0525 06:46:09.696721 139846855460672 filelock.py:318] Lock 139842528611016 released on /home/haridas/.cache/torch/transformers/90deb4d9dd705272dc4b3db1364d759d551d72a9f70a91f60e3a1f5e278b985d.9019d8d0ae95e32b896211ae7ae130d7c36bb19ccf35c90a9e51923309458f70.lock
I0525 06:46:09.697899 139846855460672 configuration_utils.py:285] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large

I0525 06:46:10.687374 139846855460672 filelock.py:274] Lock 139842528627736 acquired on /home/haridas/.cache/torch/transformers/cee054f6aafe5e2cf816d2228704e326446785f940f5451a5b26033516a4ac3d.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1.lock
I0525 06:46:10.689900 139846855460672 file_utils.py:436] https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-cased-vocab.txt not found in cache or force_download set to True, downloading to /home/haridas/.cache/torch/transformers/tmpoaseracw


I0525 06:46:12.439737 139846855460672 file_utils.py:440] storing https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-cased-vocab.txt in cache at /home/haridas/.cache/torch/transformers/cee054f6aafe5e2cf816d2228704e326446785f940f5451a5b26033516a4ac3d.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
I0525 06:46:12.441061 139846855460672 file_utils.py:443] creating metadata file for /home/haridas/.cache/torch/transformers/cee054f6aafe5e2cf816d2228704e326446785f940f5451a5b26033516a4ac3d.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
I0525 06:46:12.442286 139846855460672 filelock.py:318] Lock 139842528627736 released on /home/haridas/.cache/torch/transformers/cee054f6aafe5e2cf816d2228704e326446785f940f5451a5b26033516a4ac3d.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1.lock
I0525 06:46:12.443029 139846855460672 tokenization_utils.py:1015] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-cased-vocab.txt

I0525 06:46:13.455157 139846855460672 filelock.py:274] Lock 139842528627792 acquired on /home/haridas/.cache/torch/transformers/a473a240b2bc0d94657042a1fa78159e6a4113d009e650000fa6a9d23aa06548.8309196c7882bd29c98bab9bde57c29e2c2c75cf29d7143d1d88ce0b25eafe63.lock
I0525 06:46:13.457288 139846855460672 file_utils.py:436] https://s3.amazonaws.com/models.huggingface.co/bert/dbmdz/bert-large-cased-finetuned-conll03-english/config.json not found in cache or force_download set to True, downloading to /home/haridas/.cache/torch/transformers/tmp7rr4s7p4


I0525 06:46:14.492258 139846855460672 file_utils.py:440] storing https://s3.amazonaws.com/models.huggingface.co/bert/dbmdz/bert-large-cased-finetuned-conll03-english/config.json in cache at /home/haridas/.cache/torch/transformers/a473a240b2bc0d94657042a1fa78159e6a4113d009e650000fa6a9d23aa06548.8309196c7882bd29c98bab9bde57c29e2c2c75cf29d7143d1d88ce0b25eafe63
I0525 06:46:14.493627 139846855460672 file_utils.py:443] creating metadata file for /home/haridas/.cache/torch/transformers/a473a240b2bc0d94657042a1fa78159e6a4113d009e650000fa6a9d23aa06548.8309196c7882bd29c98bab9bde57c29e2c2c75cf29d7143d1d88ce0b25eafe63
I0525 06:46:14.494969 139846855460672 filelock.py:318] Lock 139842528627792 released on /home/haridas/.cache/torch/transformers/a473a240b2bc0d94657042a1fa78159e6a4113d009e650000fa6a9d23aa06548.8309196c7882bd29c98bab9bde57c29e2c2c75cf29d7143d1d88ce0b25eafe63.lock
I0525 06:46:14.496075 139846855460672 configuration_utils.py:285] loading configuration file https://s3.amazonaws.com/model

I0525 06:46:15.418590 139846855460672 filelock.py:274] Lock 139842528621232 acquired on /home/haridas/.cache/torch/transformers/260d6de20be053fffff53f4a6d9a1be612fbbc78418a86c6b71d34123d67cce6.455d944f3d1572ab55ed579849f751cf37f303e3388980a42d94f7cd57a4e331.lock
I0525 06:46:15.421290 139846855460672 file_utils.py:436] https://s3.amazonaws.com/models.huggingface.co/bert/dbmdz/bert-large-cased-finetuned-conll03-english/modelcard.json not found in cache or force_download set to True, downloading to /home/haridas/.cache/torch/transformers/tmp46ehzq8a


I0525 06:46:16.394998 139846855460672 file_utils.py:440] storing https://s3.amazonaws.com/models.huggingface.co/bert/dbmdz/bert-large-cased-finetuned-conll03-english/modelcard.json in cache at /home/haridas/.cache/torch/transformers/260d6de20be053fffff53f4a6d9a1be612fbbc78418a86c6b71d34123d67cce6.455d944f3d1572ab55ed579849f751cf37f303e3388980a42d94f7cd57a4e331
I0525 06:46:16.396351 139846855460672 file_utils.py:443] creating metadata file for /home/haridas/.cache/torch/transformers/260d6de20be053fffff53f4a6d9a1be612fbbc78418a86c6b71d34123d67cce6.455d944f3d1572ab55ed579849f751cf37f303e3388980a42d94f7cd57a4e331
I0525 06:46:16.397365 139846855460672 filelock.py:318] Lock 139842528621232 released on /home/haridas/.cache/torch/transformers/260d6de20be053fffff53f4a6d9a1be612fbbc78418a86c6b71d34123d67cce6.455d944f3d1572ab55ed579849f751cf37f303e3388980a42d94f7cd57a4e331.lock
I0525 06:46:16.398043 139846855460672 modelcard.py:161] loading model card file https://s3.amazonaws.com/models.huggingf

I0525 06:46:17.645102 139846855460672 filelock.py:274] Lock 139842528974272 acquired on /home/haridas/.cache/torch/transformers/4b02c1fe04cf7f7e6972536150e9fb329c7b3d5720b82afdac509bd750c705d2.6dcb154688bb97608a563afbf68ba07ae6f7beafd9bd98b5a043cd269fcc02b4.lock
I0525 06:46:17.647509 139846855460672 file_utils.py:436] https://cdn.huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english/pytorch_model.bin not found in cache or force_download set to True, downloading to /home/haridas/.cache/torch/transformers/tmplem_13i8
